In [1]:
pip install kafka-python

  Obtaining dependency information for kafka-python from https://files.pythonhosted.org/packages/75/68/dcb0db055309f680ab2931a3eeb22d865604b638acf8c914bedf4c1a0c8c/kafka_python-2.0.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/246.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/246.5 kB ? eta -:--:--
   --------- ------------------------------ 61.4/246.5 kB 1.1 MB/s eta 0:00:01
   ------------------- -------------------- 122.9/246.5 kB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 153.6/246.5 kB 1.1 MB/s eta 0:00:01
   --------------------------------- ------ 204.8/246.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 246.5/246.5 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

In [63]:
producer = KafkaProducer(bootstrap_servers=['52.86.95.232:9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [65]:
producer.send('demo_test2', value={'Name':'Ankan'})

In [24]:
df = pd.read_csv("indexProcessed.csv")

In [25]:
df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [26]:
# for _ in range(5):
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test2', value=dict_stock)
    sleep(1)#to help this small machine which don't have distributed computing , have it halt for 1 sec o9ther wise, it will break   

In [23]:
producer.flush() #clear data from kafka server

In [28]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.Collecting yfinance
  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/db/fc/10b7d339ccf6725e13408d76fb1e944f512590a949af426503c38d4af712/yfinance-0.2.41-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/6a/71/3656c00606e75e81f11721e6a1c973c3e03da8c7d8b665d20f78245384c6/frozendict-2.4.4-py311-none-any.whl.metadata
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
      --------------------------------------- 0.1/3.0 MB 3.2 MB/s eta 0:00:01
     -- ------------------------------------- 0.2/3.0 MB 2.4 MB/s eta 0:00:02
     --- ------------------------------------ 0.3/3

In [66]:
import yfinance as yf
import numpy as np
# Function to download AMZN stock data
def download_stock_data(stock_list):
    stock_data = {}
    for stock in stock_list:
        data = yf.download(stock, period="5y", interval="1d")
        if isinstance(data.index, pd.DatetimeIndex):
            data.index = data.index.tz_localize(None)  # Check index type before localizing
        data.reset_index(inplace=True)
        stock_info = yf.Ticker(stock).info
        market_cap = stock_info.get('marketCap', np.nan)
        data['Market Cap'] = market_cap
        data['Ticker'] = stock
        stock_data[stock] = data
    return stock_data

# Download AMZN stock data
stock_data = download_stock_data(['AMZN'])
dfyf = stock_data['AMZN']
# Convert Timestamp to string
dfyf['Date'] = dfyf['Date'].astype(str)
dfyf.head()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,Market Cap,Ticker
0,2019-07-24,98.464996,100.065002,98.293503,100.040497,100.040497,52626000,1939894304768,AMZN
1,2019-07-25,100.050003,100.059998,98.636002,98.691002,98.691002,82730000,1939894304768,AMZN
2,2019-07-26,97.099998,97.544998,96.225502,97.152496,97.152496,98542000,1939894304768,AMZN
3,2019-07-29,96.500000,96.611504,94.527000,95.622498,95.622498,89864000,1939894304768,AMZN
4,2019-07-30,94.556000,95.494499,94.174004,94.926498,94.926498,58218000,1939894304768,AMZN


In [67]:
dfyf.count()

Date          1258
Open          1258
High          1258
Low           1258
Close         1258
Adj Close     1258
Volume        1258
Market Cap    1258
Ticker        1258
dtype: int64

In [68]:
# for _ in range(5):
while True:
    dict_stock = dfyf.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test2', value=dict_stock)
    sleep(1)  # Help this small machine by halting for 1 second to avoid breaking

KeyboardInterrupt: 

In [69]:
# Flush the producer to ensure all messages are sent
producer.flush()

In [70]:

# Close the producer
producer.close()